In [1]:
!pip install findspark

import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext


sc = SparkContext('local')
spark = SparkSession(sc)
sc


<SparkContext master=local appName=pyspark-shell>

In [2]:
#başlıkları okusun diye header = True
# integer olanları okusun, her şeyi string olarak görmesin diye inferSchema= True
# veri setinin raw halie ";" ile ayrılmış , onun için sep argümanını kullandık
df=spark.read.csv('OnlineRetail.csv',header=True , inferSchema = True, sep = ";")

In [3]:
df.show(10)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|1.12.2010 08:26|     7,65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|1.12.2010 08:

In [4]:
#yukarıda  inferSchema = True dedigimiz icin ornegin integer olması gereken quantity sütunun 
# dogru bir sekilde quantity olarak okumuş
# printSchema() ptython'da dtypes gibi çalışır.
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [5]:
df.count ()

541909

In [6]:
#invoiceNo sütunua göre sorting yaptı.
df.sort("InvoiceNo").show(10)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|1.12.2010 08:26|     7,65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|1.12.2010 08:26|     4,25|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:

In [7]:
# sadece belirli sütunları seçtik
df.select ('InvoiceNo','StockCode').show (5)

+---------+---------+
|InvoiceNo|StockCode|
+---------+---------+
|   536365|   85123A|
|   536365|    71053|
|   536365|   84406B|
|   536365|   84029G|
|   536365|   84029E|
+---------+---------+
only showing top 5 rows



In [8]:
# veri setiyle çalışmaya karar verdiğmizde onu cashe'leyip Local'e çekmemiz gerekir
#aksi halde veri seti her seferinde server'lerden çekilmek zorunda kalır
df.cache ()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: string, CustomerID: int, Country: string]

In [9]:
# SQL  çalıştırmak üzere yeni nesne yaratacağız tablo adında
df.createOrReplaceTempView("tablo")

In [10]:
#ilk 10 satırı gördük
spark.sql(""" 
SELECT * FROM tablo 


""").show(10)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|1.12.2010 08:26|     7,65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|1.12.2010 08:

In [11]:
# group by country yaparak sql sorugusu girdik
spark.sql("""

SELECT Country, SUM(UnitPrice) UnitPrice 
FROM tablo
GROUP BY Country
ORDER BY UnitPrice DESC

""").show(20)

+---------------+---------+
|        Country|UnitPrice|
+---------------+---------+
| United Kingdom|  94911.0|
|           EIRE|   9423.0|
|        Germany|   7930.0|
|         France|   6288.0|
|          Spain|   2927.0|
|        Finland|   1578.0|
|        Belgium|   1503.0|
|         Norway|   1451.0|
|    Switzerland|   1267.0|
|         Sweden|    921.0|
|         Cyprus|    890.0|
|    Netherlands|    874.0|
|          Italy|    823.0|
|       Portugal|    817.0|
|Channel Islands|    736.0|
|        Austria|    548.0|
|      Australia|    350.0|
|        Denmark|    246.0|
|          Malta|    210.0|
|         Poland|    200.0|
+---------------+---------+
only showing top 20 rows



SQL'İ BİR KENARA BIRAKTIK

ARTIK SPARK İLE
ÖNCEDEN YÜKLEDEĞİMİZ ONLINE RETAIL VERİ SETİ ÜZERİNDEN
STRİNG İŞLEMLERE DEVAM EDİYORUZ SONRAKİ VİDEO'DA
VİDEO 82

In [12]:
# sql string işlemleri yapmak için kütüphane
from pyspark.sql.functions import*

In [13]:
# concat iki sütunu ya da satırı birleştirir
# lit ("-") ile yeni iki sütun arasında ("-") boşluk kodyduk
# sonuçta country ve description sütünlarının birleştiği yeni bir sütun elde ettik
df_concat = df\
.withColumn("newcolumns", concat(col("Country"), lit ("-"), col("Description") ))

In [14]:
#ilk 5 satırı gösterdi
# truncate=True  dersek bütün sütünları sayfaya sıkıştırarak verir.aksi halde sütunları bütün genişliği ile verir.
df_concat.show (n=5, truncate = True)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+--------------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|          newcolumns|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+--------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|United Kingdom-WH...|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|United Kingdom-WH...|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|United Kingdom-CR...|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|United Kingdom-KN...|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|United Kingdom-RE...|


In [15]:
# number format ile numara formatlıyarak quantity sütununa virgül atacağız, lazım değil ama örnek için
#df_number ile yeniden atadık
# sonuç olarak new quantity adında yeni bir sütun oluştu ve iki haneli ondalık sayıları var.
df_number = df\
df_number.show (n=5, truncate = True)
#truncate= true ile sonuç ekranını tam olarak ekrana sığdırdık

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|new-quantity|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|        6.00|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|        6.00|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|        8.00|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|        6.00|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|        6.00|
+---------+---------+--------------------+--------+-------------

LOWERCASE UPPER CASE KULLANIMI

In [16]:
#country sütunundaki bütün ülkelerin isimlerini lowercase yaptık
#description sütunun bütün isimlerin ilk harflerini büyük yaptık.
df.select("Description","Country")\
.withColumn("Country", lower(col("Country"))) \
.withColumn("Description", initcap(col("Description"))) \
.show(5)

+--------------------+--------------+
|         Description|       Country|
+--------------------+--------------+
|White Hanging Hea...|united kingdom|
| White Metal Lantern|united kingdom|
|Cream Cupid Heart...|united kingdom|
|Knitted Union Fla...|united kingdom|
|Red Woolly Hottie...|united kingdom|
+--------------------+--------------+
only showing top 5 rows



VERİ TEMİZLİĞİ İÇİN BAŞKA SİMPLEDATA ADINDA YENİ BİR VERİ SETİ YÜKLEDİK


In [17]:
df=spark.read.csv('simpledata.csv',header=True , inferSchema = True, sep = ",")

In [18]:
df.show (5)

+------+-------+---+-----------+--------+-----------+
|sirano|   isim|yas|     meslek|   sehir|aylik_gelir|
+------+-------+---+-----------+--------+-----------+
|     1|  Cemal| 35|       Isci|  Ankara|       3500|
|     2|  Ceyda| 42|      Memur| Kayseri|       4200|
|     3|  Timur| 30|  MÃ¼zisyen|Istanbul|       9000|
|     4|  Burcu| 29|Pazarlamaci|  Ankara|       4200|
|     5|Yasemin| 23|Pazarlamaci|   Bursa|       4800|
+------+-------+---+-----------+--------+-----------+
only showing top 5 rows



In [19]:
# sql string işlemleri yapmak için kütüphane yükledik artık bundan sonra sql işlemlerini F olarak çağıracağız 
from pyspark.sql import functions as F

veri temizleme, düzeltme işlemleri.

boşlukları trim yaptık

null değerleri doldurdurk

şehirleri upper yaptık

In [20]:
df2 = df \
.withColumn("isim", F.trim(F.initcap(df.isim))) \
.withColumn("meslek", F.when(df['meslek'].isNull(), "U").otherwise(df['meslek'])) \
.withColumn("sehir", F.when(df['sehir'].isNull(), "BİLİNMİYOR").otherwise(F.trim(F.upper(df['sehir']))))


df2.show(15)

+------+--------+---+-----------+--------+-----------+
|sirano|    isim|yas|     meslek|   sehir|aylik_gelir|
+------+--------+---+-----------+--------+-----------+
|     1|   Cemal| 35|       Isci|  ANKARA|       3500|
|     2|   Ceyda| 42|      Memur| KAYSERI|       4200|
|     3|   Timur| 30|  MÃ¼zisyen|ISTANBUL|       9000|
|     4|   Burcu| 29|Pazarlamaci|  ANKARA|       4200|
|     5| Yasemin| 23|Pazarlamaci|   BURSA|       4800|
|     6|     Ali| 33|      Memur|  ANKARA|       4250|
|     7|   Dilek| 29|Pazarlamaci|ISTANBUL|       7300|
|     8|   Murat| 31|  MÃ¼zisyen|ISTANBUL|      12000|
|     9|   Ahmet| 33|     Doktor|  ANKARA|      18000|
|    10|Muhittin| 46|     Berber|ISTANBUL|      12000|
|    11|Hicaziye| 47| Tuhafiyeci|  ANKARA|       4800|
|    12|   Harun| 43|   TornacÄ±|  ANKARA|       4200|
|    13|  Hakkä±| 33|      Memur|  Ã‡ORUM|       3750|
|    14|Gã¼lizar| 37|     Doktor|  Ä°ZMIR|      14250|
|    15| Åehmuz| 41|  MÃ¼zisyen|  ANKARA|       8700|
+------+--

In [24]:
df2.coalesce(1).toPandas().to_csv('mycsv.csv')

Temizlenmiş dataframe'i diske yaz


Diske yazılmış temiz csv dosyasını okuma

In [25]:
df3=spark.read.csv('mycsv.csv',header=True , inferSchema = True, sep = ",")
df3.show(15)

+---+------+--------+---+-----------+--------+-----------+
|_c0|sirano|    isim|yas|     meslek|   sehir|aylik_gelir|
+---+------+--------+---+-----------+--------+-----------+
|  0|     1|   Cemal| 35|       Isci|  ANKARA|       3500|
|  1|     2|   Ceyda| 42|      Memur| KAYSERI|       4200|
|  2|     3|   Timur| 30|  MÃ¼zisyen|ISTANBUL|       9000|
|  3|     4|   Burcu| 29|Pazarlamaci|  ANKARA|       4200|
|  4|     5| Yasemin| 23|Pazarlamaci|   BURSA|       4800|
|  5|     6|     Ali| 33|      Memur|  ANKARA|       4250|
|  6|     7|   Dilek| 29|Pazarlamaci|ISTANBUL|       7300|
|  7|     8|   Murat| 31|  MÃ¼zisyen|ISTANBUL|      12000|
|  8|     9|   Ahmet| 33|     Doktor|  ANKARA|      18000|
|  9|    10|Muhittin| 46|     Berber|ISTANBUL|      12000|
| 10|    11|Hicaziye| 47| Tuhafiyeci|  ANKARA|       4800|
| 11|    12|   Harun| 43|   TornacÄ±|  ANKARA|       4200|
| 12|    13|  Hakkä±| 33|      Memur|  Ã‡ORUM|       3750|
| 13|    14|Gã¼lizar| 37|     Doktor|  Ä°ZMIR|      1425

## ManualSchema İŞLEMLERİ ONLINE RETAIL VERİ SETİ ÜZERİNDEN

In [26]:
df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",";") \
.csv("OnlineRetail.csv")

In [27]:
df.show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows



In [28]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [ ]:
#Örneğin yukarıda UnitPrice'ın Float olmasına ihtiyaç var, ÇÜNKÜ STRING OLARAK GİRECEĞİZ
# bu sorunu düzeltmemiz için kendi şemamızı yapalım Önce gerekli kütüphaneleri yükleyelim

# yani bu sefer hazır kullanınılan "inferSchema" argümanı yerine kendi oluşturacağınız manuel schema kullanacağız

In [31]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType, DateType,DoubleType

# Elle şema oluşturma

In [32]:
manual_schema = StructType([
    StructField("InvoiceNo", StringType(), True),
    StructField("StockCode", StringType(), True),
    StructField("Description", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("InvoiceDate", StringType(), True),
    StructField("UnitPrice", FloatType(), True),
    StructField("CustomerID", IntegerType(), True),
    StructField("Country", StringType(), True)

])

## Veriye elle hazırlanan şema ile tekrar okuma

In [33]:
df2 = spark.read \
.option("header","True") \
.option("sep",";") \
.schema(manual_schema) \
.csv("OnlineRetail.csv")

df2.show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     null|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     null|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     null|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     null|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     null|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows



In [ ]:
# unit price sütunu NULL OLARAK çıktı bu sorunu çözeceğiz 
#UnitPrice'da sıkıntı var. "." yerine "," olduğu için schma string'den float'a dönüştüremiyor. Çözüm bulmak gerekli.

### ÇÖZÜM

Çözüm olarak veriyi okurken "," ile "." yı değiştirerek okuyalım ve dataframe'i tekrar , ve . değişmiş şekilde diske yazalım. Diskten okurken düzeltilmiş csv den okuyalım ve elle hazırlanmış şemamızı kullanalım.

## Veriyi temizleyerek okuma

In [ ]:
#EN ALTTAKİ SATIRDA VİRGÜL YERİNE ARTIK NOKTA GELSİN DİYE UNİT PRİCE ÜZERİNDE DEĞİŞİKLİK YAPTIK

In [34]:
from pyspark.sql import functions as F

df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",";") \
.csv("OnlineRetail.csv") \
.withColumn("UnitPrice",F.regexp_replace(F.col("UnitPrice"), ",","."))

In [ ]:
#ŞİMDİ DİSKE YAZIYORUZ

In [38]:
df.coalesce(1).toPandas().to_csv('onlineretails.csv')

### Temizlenmiş olarak diske yazdığımız dosyayı tekrar okuyalım. Bu sefer manual_schema verelim.

In [40]:
df_temiz=spark.read.csv('onlineretails.csv',header=True , inferSchema = True, sep = ",")

In [41]:
df_temiz.show(5)

+---+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|_c0|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|  0|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2.55|     17850|United Kingdom|
|  1|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
|  2|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2.75|     17850|United Kingdom|
|  3|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
|  4|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
+---+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows



In [42]:
sc.stop()